In [1]:
import sys
sys.path.append('..')

from transport_frames.graphbuilder.graph import Graph 
from transport_frames.frame_grader.advanced_grade import AdvancedGrader
from transport_frames.framebuilder.frame import Frame
import transport_frames.utils.helper_funcs as he
from transport_frames.indicators.indicator_terr import indicator_territory
from transport_frames.indicators.indicator_area import indicator_area, preprocess_service_accessibility
from transport_frames.indicators.utils import availability_matrix, create_service_dict
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely import Polygon, MultiPolygon
import shapely
from iduedu import get_adj_matrix_gdf_to_gdf
from iduedu import get_intermodal_graph

/Users/polina/Desktop/github/transport_frames/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Graph creation

In [2]:
geocode=81993
polygon = ox.geocode_to_gdf(f'R{geocode}',by_osmid=True)
local_crs = 32637

In [3]:
# g = Graph.from_polygon(polygon, crs=local_crs)

tula = he.convert_geometry_from_wkt(nx.read_graphml('/Users/polina/Desktop/clone_latest/transport_frames/notebooks/tula.graphml'))
tula = he.convert_list_attr_from_str(tula)
g =  Graph(tula,crs=local_crs)

2025-06-23 12:52:57.860 | INFO     | transport_frames.graphbuilder.graph:_prepare_attrs:145 - Preparing the graph...
2025-06-23 12:53:07.334 | INFO     | transport_frames.graphbuilder.graph:_prepare_attrs:183 - Graph is ready!


## Frame creation

In [4]:
# polygon regions cities
regions = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/regions_of_russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions[regions['geometry'].apply(lambda x: isinstance(x, (Polygon, MultiPolygon)))]

admin_centers = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
admin_centers = admin_centers[admin_centers.is_admin_centre_district==1][['name','geometry']].reset_index(drop=True)


In [5]:
f = Frame(g.graph, regions, polygon, admin_centers,country_polygon=gpd.read_parquet('rus.parquet'))

In [6]:
import numpy as np
import shapely
print("NumPy version:", np.__version__)
print("Shapely version:", shapely.__version__)

NumPy version: 2.1.3
Shapely version: 2.1.1


## Grading custom territory (frame-based)

In [7]:
custom_terr = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson').iloc[[5]]

In [8]:
graded_terr = f.grade_territory(f.frame, custom_terr)
graded_terr

,name,geometry,grade
5,Белев,"POLYGON ((308350.059 5967359.728, 308519.156 5...",5


## Criteria of the territory


In [9]:
settlement_points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
settlement_polygons = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')
railway_stations = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
bus_stops = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
aero = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')

In [10]:
# inter = get_intermodal_graph(osm_id=geocode,
#                         keep_routes_geom=True,clip_by_bounds=True)
# # adj_drive = availability_matrix(g.graph, settlement_points,settlement_points,local_crs=local_crs)
# adj_inter = availability_matrix(inter, settlement_points, settlement_points, local_crs=local_crs)

In [11]:
import pandas as pd
adj_drive=pd.read_json('drive.json')
adj_inter=pd.read_json('inter.json')

In [12]:
grader = AdvancedGrader(local_crs)
cri = grader.get_criteria(graded_terr=graded_terr,
                          points=settlement_points,
                          polygons=settlement_polygons,
                          r_stops=railway_stations,
                          b_stops=bus_stops,
                          aero=aero,
                          adj_mx_drive=adj_drive,
                          adj_mx_inter=adj_inter)
cri


,geometry,name,grade,weight,weight_r_stops,weight_b_stops,weight_ferry,weight_aero,car_access_quartile,public_access_quartile,car_grade,public_transport_grade,overall_assessment
0,"POLYGON ((308350.059 5967359.728, 308519.156 5...",Белев,5,0.7,0.35,0.35,0.0,0.0,4,4,2.0,1.4,1.7


In [13]:
grader.interpret_gdf(cri)

[('Белев',
  ['Территория расположена в непосредственной близости от одной из региональных трасс (в радиус 5 км от границ территории попадает хотя бы 1 узел региональной трассы, являющейся приоритетной, а ближайший федеральный узел находится в не более чем 100км);',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на личном транспорте;',
   'Территория попадает в IV квартиль связности (худшие 25% МО) на общественном транспорте;',
   'В радиусе 15 км отсутствуют порты/причалы/переправы, аэродромы.'])]

## Indicators of the territory (graph-based)

In [14]:
# services
railway_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway_station.geojson')
railway_roads = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/railway.geojson')
bus_stops = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/bus_stop.geojson')
bus_routes = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/preprocessed/Тульская_область/Тульская_область_bus_routes.geojson')
bus_routes['route'] = bus_routes['desc'] 
fuel_stations = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия fuel.geojson')
local_aerodrome = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/aerodrome.geojson')
local_aerodrome['geometry'] = shapely.centroid(local_aerodrome['geometry']).set_crs(local_aerodrome.crs)
international_aerodrome = local_aerodrome[local_aerodrome['aerodrome:type']=='international']
international_aerodrome['geometry'] = shapely.centroid(international_aerodrome['geometry']).set_crs(international_aerodrome.crs)

# nature objects
water = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия water.geojson')
nature_reserve = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Копия nature_reserve.geojson')

capital =ox.geocode_to_gdf('N34389350',by_osmid=True)
districts_polygons = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/district.geojson')
settlement_polygons = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/settlement.geojson')
settlement_points = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')

In [15]:
services = create_service_dict(
                                railway_stations=railway_stations,
                                railway_paths=railway_roads,
                                bus_stops=bus_stops,
                                bus_routes=bus_routes,
                                fuel_stations=fuel_stations,
                                local_aerodrome=local_aerodrome,
                                international_aerodrome=international_aerodrome,
                                water_objects=water,
                                nature_reserve=nature_reserve,          
                                local_crs=local_crs)

In [16]:
t = indicator_territory(g.graph,custom_terr,services,capital,admin_centers,settlement_points,districts_polygons,local_crs,)
t

2025-06-23 12:54:11.201 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:27 - Graph contains 17 weakly connected components. This means the graph has disconnected groups if edge directions are ignored. Component sizes:: [30440, 16, 61, 2, 68, 2, 2, 2, 3, 2, 2, 2, 6, 2, 2, 2, 13]
2025-06-23 12:54:11.250 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:37 - Removing 216 nodes from 31 smaller strongly connected components. These are subgraphs where nodes are internally reachable but isolated from the rest. Retaining only the largest strongly connected component (30411 nodes).
2025-06-23 12:54:12.289 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:27 - Graph contains 17 weakly connected components. This means the graph has disconnected groups if edge directions are ignored. Component sizes:: [30440, 16, 61, 2, 68, 2, 2, 2, 3, 2, 2, 2, 6, 2, 2, 2, 13]
2025-06-23 12:54:12.339 | WARNING  | iduedu.utils.utils:keep_larg

,name,geometry,to_region_admin_center_km,to_reg_1_km,fuel_stations_accessibility_min,number_of_fuel_stations,local_aerodrome_accessibility_min,number_of_local_aerodrome,international_aerodrome_accessibility_min,number_of_international_aerodrome,...,number_of_bus_stops,number_of_water_objects,water_objects_accessibility_min,number_of_nature_reserve,nature_reserve_accessibility_min,train_path_length_km,number_of_bus_routes,to_nearest_district_center_km,to_nearest_settlement_km,road_density_km/km2
0,Белев,"POLYGON ((305406.67 5964767.649, 305378.241 59...",85.94,0.0,0.0,3.0,113.9,0.0,None,0,...,20.0,32,0.0,0,89.54,14.844,0,22.103,9.386,1.483


## Indicators of the area (graph-based)

In [17]:
#area polygons
country_polygon = ox.geocode_to_gdf('RUSSIA')
country_polygon['status'] = 'country'
country_polygon['layer'] = 'country'

In [18]:
preprocessed_accessibility = preprocess_service_accessibility(settlement_points,
                                     services, 
                                     g.graph, 
                                     local_crs)

2025-06-23 12:54:18.545 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:27 - Graph contains 17 weakly connected components. This means the graph has disconnected groups if edge directions are ignored. Component sizes:: [30440, 16, 61, 2, 68, 2, 2, 2, 3, 2, 2, 2, 6, 2, 2, 2, 13]
2025-06-23 12:54:18.594 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:37 - Removing 216 nodes from 31 smaller strongly connected components. These are subgraphs where nodes are internally reachable but isolated from the rest. Retaining only the largest strongly connected component (30411 nodes).
/Users/polina/Desktop/github/transport_frames/venv/lib/python3.10/site-packages/pandas/core/construction.py:817: RuntimeWarning: overflow encountered in cast
  subarr = np.asarray(arr, dtype=dtype)
2025-06-23 12:54:19.570 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:27 - Graph contains 17 weakly connected components. This means the graph h

In [22]:
ind_area = indicator_area(g.graph, [country_polygon, districts_polygons,settlement_polygons],preprocessed_accessibility,services,local_crs,adj_drive,adj_inter,capital)

2025-06-23 12:55:00.384 | INFO     | transport_frames.indicators.indicator_area:indicator_area:178 - Calculating service accessibility
2025-06-23 12:55:00.721 | INFO     | transport_frames.indicators.indicator_area:indicator_area:195 - Calculating distance to region admin center and federal roads
2025-06-23 12:55:01.068 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:27 - Graph contains 17 weakly connected components. This means the graph has disconnected groups if edge directions are ignored. Component sizes:: [30440, 16, 61, 2, 68, 2, 2, 2, 3, 2, 2, 2, 6, 2, 2, 2, 13]
2025-06-23 12:55:01.120 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:37 - Removing 216 nodes from 31 smaller strongly connected components. These are subgraphs where nodes are internally reachable but isolated from the rest. Retaining only the largest strongly connected component (30411 nodes).
2025-06-23 12:55:01.674 | WARNING  | iduedu.utils.utils:keep_largest_str

In [23]:
ind_area[0]

,name,geometry,number_of_railway_stations,railway_stations_accessibility_min,number_of_fuel_stations,fuel_stations_accessibility_min,number_of_ports,ports_accessibility_min,number_of_local_aerodrome,local_aerodrome_accessibility_min,...,connectivity_drive_min,connectivity_inter_min,to_region_admin_center_km,to_reg_1_km,train_path_length_km,number_of_bus_routes,reg1_length_km,reg2_length_km,reg3_length_km,road_density_km/km2
0,Russia,"MULTIPOLYGON (((2408972.289 12450129.16, 24081...",63,18.15625,232,12.953125,0,NaN,4,54.9375,...,91.909,1054.546,3951.651,3832.796,2371.024,75,997.812,4212.452,11791.051,0.0


In [21]:
# restricted_terr = [(ox.geocode_to_gdf('Estonia'), 0.0),
#                    (ox.geocode_to_gdf('Finland'), 0.5)]